Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [US Dept of Treasury](https://home.treasury.gov/policy-issues/coronavirus/assistance-for-state-local-and-tribal-governments/emergency-rental-assistance-program).
<hr>

# States Emergency Rental Assistance Program

During the COVID-19 pandemic 2 programs were enacted to provide rental assistance to help people stay in their homes during the crisis. ERA1 provides up to 25 billion under the Consolidated Appropriations Act, 2021, which was enacted on December 27, 2020. The funds are provided directly to states, U.S. territories, local governments.

This Notebook code is created after the `state-rental-assistance` notebook and uses a function to ease the collection of the data. This function can then be used for both ERA1 and ERA2 excel sheets to extract the data for usage in Tableau.

<hr>

In [1]:
import pandas as pd
import json

In [2]:
remove = ['American Samoa', 'Guam', 'Puerto Rico', 'Northern Mariana Islands', 'Virgin Islands']

with open('./data/files/states.json', 'r') as f:
    states = json.load(f)['states']
    names = list(states.keys())
    names = [x for x in states if x not in remove]

In [3]:
def parse_df(df, rename_cols, assist_cols, house_cols, program_type):
    """ Function to extract state totals & state gov numbers from
        ERA1 and ERA2 rental assistance programs.
    """
    # add grand totals
    df['total_assistance'] = df[assist_cols].sum(axis=1)
    df['total_households'] = df[house_cols].sum(axis=1)
    
    # rename and keep only wanted columns
    df = df.rename(columns=rename_cols)
    df = df[['location', 'total_assistance', 'total_households', 'total_allocation']]
    
    # get percent of total allocated spent & not spent total
    df['pct_spent'] = round(df['total_assistance'] / df['total_allocation'], 4)
    df['not_spent'] = df['total_allocation'] - df['total_assistance']
    
    # get state total rows
    state_total = df[df['location'].isin(names)].copy()
    state_total['type'] = 'state_total'
    
    # get only state government rows (no local county/city)
    state_only = df[df['location'].str.contains('^State of|Common')].copy()
    state_only['type'] = 'state_only'
    
    # combine datasets, remove prefix, add abbr
    df = pd.concat([state_only, state_total]).reset_index(drop=True)
    df['location'] = df['location'].str.replace('State of ', '').str.replace('Commonwealth of ', '')
    df['state_abbr'] = df['location'].map(states)
    
    # replace 1 of the dc duplicates
    idx = df[df['state_abbr'] == 'DC'].iloc[0].name
    df.loc[df.index == idx, 'type'] = 'state_only'
    
    # add program type
    df['program'] = program_type
    
    return df

In [4]:
# collect era1
df = pd.read_excel('./data/raw/August-ERA-Data.xlsx', sheet_name='ERA1 State & Local Reporting', skipfooter=6, skiprows=2)

rename_cols = {
    'Unnamed: 0': 'location',
    'Unnamed: 14': 'total_allocation'
}

assist = list(df)[1:7]
house = list(df)[8:14]

era1 = parse_df(df, rename_cols, assist, house, "ERA1")

In [5]:
# collect era2
df = pd.read_excel('./data/raw/August-ERA-Data.xlsx', sheet_name='ERA2 State & Local Reporting', skipfooter=5, skiprows=2)

rename_cols = {
    'Unnamed: 0': 'location',
    'Unnamed: 8': 'total_allocation'
}

assist = list(df)[1:4]
house = list(df)[5:8]

era2 = parse_df(df, rename_cols, assist, house, "ERA2")

In [6]:
# combine era1 & era2
df = era1.append(era2)

# get columns for combined totals of both era1 + era2
merge = df.groupby(['location', 'state_abbr', 'type'], as_index=False).sum()
merge['pct_spent'] = round(merge['total_assistance'] / merge['total_allocation'], 4)
merge['not_spent'] = merge['total_allocation'] - merge['total_assistance']
merge['program'] = 'combined'

# final output
df.append(merge).to_csv('./data/clean/era1_and_era2_clean.csv', index=False)